# Sqlite database
- https://www.sqlite.org/
- C-based, one of the most used embedded database (zero configuration)

## sqlite browser
- http://sqlitebrowser.org/

## sql basics
 - http://www.w3schools.com/sql/default.asp

## sqlite3 module
- https://docs.python.org/3/library/sqlite3.html
- SQLite natively supports the following types: NULL, INTEGER, REAL, TEXT, BLOB 
<table>
<tr><td>Python type</td>
    <td>SQLite type</td>
</tr>
<tr><td>None</td>
    <td>NULL</td>
</tr>
<tr><td>int</td>
    <td>INTEGER</td>
</tr>
<tr><td>float</td>
    <td>REAL</td>
</tr>
<tr><td>str</td>
    <td>TEXT</td>
</tr>
<tr><td>bytes</td>
    <td>BLOB</td>
</tr>
</table>

## in memory db example

In [ ]:
import sqlite3
# connect to the memory database
con = sqlite3.connect(":memory:")

# create a table
con.execute("create table person(fname, lname)")

In [ ]:
# fill the table with data
persons = [('Hugo', 'Boss'), ('Calvin', 'Klien')]
con.executemany("insert into person(fname, lname) values (?, ?)", 
                persons)

In [ ]:
# print the table contents
for row in con.execute("select rowid, fname, lname from person"):
    print(row)

In [ ]:
print("I just deleted", con.execute("delete from person where rowid=1").rowcount, 
      "rows")

## db file example
### create database, create table and insert data into table

In [2]:
import sqlite3
# create connection
conn = sqlite3.connect('example.db')
# create cursor object
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS students (
                firstName text, 
                lastName text, 
                test1 real, 
                test2 real,
                average real,
                grade text
                )
            """)

In [3]:
query = """ INSERT INTO students (firstName, lastName, 
            test1, test2) values (?, ?, ?, ?)
        """
cur.execute(query, ('John', 'Smith', 99, 95.5))

In [4]:
cur.execute(query, ('Michael', 'Jordan', 50, 65))

In [5]:
# save/commit the changes to the db
conn.commit()
# close the database if done
conn.close()

### open database, read and update table

In [6]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

In [7]:
cur.execute('SELECT * FROM students where rowid = 1')
row = cur.fetchone() # returns one row as tuple if rowid with value 1 exists
print(row)

('John', 'Smith', 99.0, 95.5, None, None)


In [8]:
for col in row:
    print(col)

John
Smith
99.0
95.5
None
None


In [9]:
cur.execute('SELECT rowid, * FROM students')
rows = cur.fetchall()
print(type(rows))

<class 'list'>


In [10]:
for row in rows:
    print(row)

(1, 'John', 'Smith', 99.0, 95.5, None, None)
(2, 'Michael', 'Jordan', 50.0, 65.0, None, None)


<strong>update table</strong>

In [11]:
for row in rows:
    avg = (row[3] + row[4])/2
    # grade = ?
    cur.execute('update students set average=? where rowid=?', (avg, row[0]))

In [12]:
cur.execute('select * from students')
print(cur.fetchall())

[('John', 'Smith', 99.0, 95.5, 97.25, None), ('Michael', 'Jordan', 50.0, 65.0, 57.5, None)]


In [13]:
# commit changes and close connection
conn.commit()
conn.close()

## SQL Injection Vulnerability
- how not to write sql query in programs

In [14]:
import sqlite3
conn = sqlite3.connect('example.db')
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS users (
                username text unique, 
                password text 
                )
            """)

In [15]:
# Prompt user to create account
username = input('Enter your username: ')
password = input('Pick a password: ')

Enter your username: john
Pick a password: P@$sw0rd!


In [16]:
# bad passwords
# insecure way to create sql statements
sqlinsert = """insert into users (username, password) values 
            ('{0}', '{1}')
            """.format(username, password)

cur.execute(sqlinsert)


In [17]:
# check database
conn.commit()
for row in cur.execute('select * from users'):
    print(row)

('john', 'P@$sw0rd!')


### what is wrong with the above codes?

### verify users and SQL injection attack

In [18]:
# Prompt user to create account
def insecureAuthentication():
    username = input('Enter your username: ')
    password = input('Pick a password: ')
    sqlSelect = '''select * from users where username = '{0}' 
                    and password = '{1}'
                '''.format(username, password)
    cur.execute(sqlSelect)
    row = cur.fetchone()
    if row:
        print('Authentication sucessful! Welcome to your kingdom!')
    else:
        print('Wrong credentials. Try Again!')
        

In [22]:
insecureAuthentication()

Enter your username: john
Pick a password: ' or '1'='1
Authentication sucessful! Welcome to your kingdom!


## secure way to store password
- https://docs.python.org/3/library/hashlib.html


In [23]:
import uuid
import hashlib, binascii

def createSecurePassword(password, salt=None, round=100000):
    if not salt:
        salt = uuid.uuid4().hex
    
    """
    for i in range(round):
        password = password+salt
        password = hashlib.sha256(password.encode('utf-8')).hexdigest()
    """
    # hashlib.pbkdf2_hmac(hash_name, password, salt, iterations, dklen=None)
    dk = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), 
                        salt.encode('utf-8'), round)
    password = binascii.hexlify(dk)
    return "%s:%s"%(password, salt)

In [24]:
def secureRegistration():
    # Prompt user to create account
    username = input('Enter your username: ')
    password = input('Pick a password: ')
    secPass = createSecurePassword(password)
    insert = '''insert into users (username, password)
                values (?, ?)
                '''
    cur.execute(insert, (username, secPass))
    

In [28]:
# register a user
secureRegistration()

Enter your username: rbasnet
Pick a password: P@$sw0rd


In [29]:
# check data
for row in cur.execute('select * from users'):
    print(row)

('john', 'P@$sw0rd!')
('jake', "b'2e961444d35fd0f80b24806fc0b9c2b8eee26a0647db12470da00d6bba61f5a7':bddf07e55c154eb78b01145a284f9603")
('rbasnet', "b'ccbc5eef70a3e914bf78e2f9ae89db36a7d473ad4175062ef469f55429a45573':03d7d17444e2454789285dc0de92198e")


In [27]:
conn.commit()

In [31]:
def secureAuthentication():
    username = input('Enter your username: ')
    password = input('Enter your password: ') 
    # use parameterized query
    sqlSelect = 'select password from users where username = ?'
    cur.execute(sqlSelect, (username,))
    row = cur.fetchone()
    if row:
        # username exists
        # check password hashes
        hashpass = row[0]
        hashedPass = hashpass[:hashpass.find(':')]
        salt = hashpass[hashpass.find(':')+1:]
        secPass = createSecurePassword(password, salt)
        if hashpass == secPass:
            print('Authentication sucessful! Welcome to your kingdom...')
        else:
            print('Wrong credentials. Try Again!')
    else:
        print('Wrong credentials. Try Again!')

In [34]:
secureAuthentication()

Enter your username: jake
Enter your password: ' or '1'='1
jake
Wrong credentials. Try Again!


In [35]:
conn.commit()
conn.close()